In [ ]:
# import all the required packages
from google.colab import files 
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd 
import math 
import tensorflow as tf 
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Dense, LSTM 
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error

## Data preparation

In [ ]:
uploaded = files.upload()

Saving Dataset_BF_2015.csv to Dataset_BF_2015.csv


In [ ]:
uploaded_1 = files.upload()

Saving Dataset_BF_2016.csv to Dataset_BF_2016.csv


In [ ]:
uploaded_2 = files.upload()

Saving Dataset_BF_2017.csv to Dataset_BF_2017.csv


In [ ]:
import pandas as pd 
import io 

data = pd.read_csv(io.StringIO(uploaded['Dataset_BF_2015.csv'].decode('utf-8')),sep=',')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 411 entries, 0 to 410
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Dte             411 non-null    object 
 1   number_clients  411 non-null    int64  
 2   Shift           411 non-null    int64  
 3   day             411 non-null    object 
 4   Holiday         11 non-null     object 
 5   Before Holiday  8 non-null      float64
 6   Unnamed: 6      1 non-null      object 
dtypes: float64(1), int64(2), object(4)
memory usage: 22.6+ KB


In [ ]:
data_1= pd.read_csv(io.StringIO(uploaded_1['Dataset_BF_2016.csv'].decode('utf-8')),sep=',')
data_1.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 415 entries, 0 to 414
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Dte             415 non-null    object 
 1   number_clients  415 non-null    int64  
 2   Shift           415 non-null    int64  
 3   day             415 non-null    object 
 4   Holiday         8 non-null      object 
 5   Before Holiday  4 non-null      float64
dtypes: float64(1), int64(2), object(3)
memory usage: 19.6+ KB


In [ ]:
data_2= pd.read_csv(io.StringIO(uploaded_2['Dataset_BF_2017.csv'].decode('utf-8')),sep=',')
data_2.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 414 entries, 0 to 413
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Dte             414 non-null    object 
 1   number_clients  414 non-null    int64  
 2   Shift           414 non-null    int64  
 3   day             414 non-null    object 
 4   Holiday         11 non-null     object 
 5   Before holiday  8 non-null      float64
dtypes: float64(1), int64(2), object(3)
memory usage: 19.5+ KB


In [ ]:
# combine the different data set 
data = pd.concat([data,data_1],sort=False,ignore_index=True)
data = pd.concat([data,data_2],sort=False, ignore_index=True )
data.info() 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1240 entries, 0 to 1239
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Dte             1240 non-null   object 
 1   number_clients  1240 non-null   int64  
 2   Shift           1240 non-null   int64  
 3   day             1240 non-null   object 
 4   Holiday         30 non-null     object 
 5   Before Holiday  12 non-null     float64
 6   Unnamed: 6      1 non-null      object 
 7   Before holiday  8 non-null      float64
dtypes: float64(2), int64(2), object(4)
memory usage: 77.6+ KB


In [ ]:
#select just the number of clients variable
data_ts = data.iloc[:,[1]]
data_ts.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1240 entries, 0 to 1239
Data columns (total 1 columns):
 #   Column          Non-Null Count  Dtype
---  ------          --------------  -----
 0   number_clients  1240 non-null   int64
dtypes: int64(1)
memory usage: 9.8 KB


In [ ]:
#transform into float. I read that the algorithm prefers float number 
data_ts = data_ts.astype('float32')
data_ts.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1240 entries, 0 to 1239
Data columns (total 1 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   number_clients  1240 non-null   float32
dtypes: float32(1)
memory usage: 5.0 KB


In [ ]:
# normalize the dataset
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0, 1))
data_ts_norm = scaler.fit_transform(data_ts) #attention transform into a numpy array 

## Training, test set split and reshape to meet the requirement structure to feed the neural network 

In [ ]:
#separate into training and test set 
train_size = int(len(data_ts_norm) * 0.8)
test_size = len(data_ts_norm) - train_size
train, test = data_ts_norm[0:train_size,:], data_ts_norm[train_size:len(data_ts_norm),:] 

In [ ]:
#create dataset with a certain lag (= look_back)
def create_dataset (dataset,look_back =1):
  dataX, dataY= [],[]
  for i in range(len(dataset)-look_back-1):
    a= dataset[i:(i+look_back),0]
    dataX.append(a)
    dataY.append(dataset[i+look_back,0])
  return np.array(dataX), np.array(dataY)

In [ ]:
#create the training and test set
trainX, trainY = create_dataset(train)
testX, testY = create_dataset(test)

In [ ]:
# the model needs the data in form of a certain structure [samples, time steps, features]
trainX = np.reshape(trainX,(trainX.shape[0],1,trainX.shape[1]))
testX = np.reshape(testX,(testX.shape[0],1,testX.shape[1]))

## Model training and assesment 

In [ ]:
# Model construction 
model = Sequential()
model.add(LSTM(128, input_shape=(trainX.shape[1:]), activation= 'relu',return_sequences=True))
model.add(LSTM(128,activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(9,activation='relu'))
model.add(Dense(1,activation='relu'))

opt= tf.keras.optimizers.Adam(lr=1e-3, decay= 1e-5) #use the optimizer Adam  

# training and fit
model.compile(loss='mean_absolute_error', optimizer=opt, metrics=['accuracy'])
model.fit(trainX,trainY, epochs=100, batch_size=1, verbose=2)

In [ ]:
# Use the model for the prediction for computation of the MAE 
trainPredict = model.predict(trainX)
testPredict = model.predict(testX)

In [ ]:
# Reverse the normalisation transformation to get the on scale data
trainPredict = scaler.inverse_transform(trainPredict)
trainY = scaler.inverse_transform([trainY])
testPredict = scaler.inverse_transform(testPredict)
testY = scaler.inverse_transform([testY])

In [ ]:
# score computation of the training and test set 
trainScore = mean_absolute_error(trainY[0], trainPredict[:,0])
print('Train Score: %.2f MAE' % (trainScore))
testScore = mean_absolute_error(testY[0], testPredict[:,0])
print('Test Score: %.2f MAE' % (testScore))

Train Score: 17.22 MAE
Test Score: 18.11 MAE


From here, it is trial code. Don't pay attention to it !

In [ ]:
def ts2sp (data, lag=1):
  df= pd.DataFrame(data)
  columns= [df.shift(i) for i in range(1, lag+1)]
  columns.append(df)
  df = pd.concat(columns,axis=1)
  df.fillna(0, inplace=True)
  return df 

In [ ]:
data_supervised = ts2sp (data_ts_norm,1)
data_supervised.info()
data_supervised.head()

In [ ]:
train,test= data_supervised[0:-9], data_supervised[-9:]

In [ ]:
def fit_lstm(train, batch_size, nb_epoch, neurons):
	X, y = train[:, 0:-1], train[:, -1]
	X = X.reshape(X.shape[0], 1, X.shape[1])
	model = Sequential()
	model.add(LSTM(neurons, batch_input_shape=(batch_size, X.shape[1], X.shape[2]), stateful=True))
	model.add(Dense(1))
	model.compile(loss='mean_asbolute_error', optimizer='adam')
	for i in range(nb_epoch):
		model.fit(X, y, epochs=1, batch_size=batch_size, verbose=0, shuffle=False)
		model.reset_states()
	return model

In [ ]:
model = fit_lstm(train,1,100,4)